# 示例：使用朴素贝叶斯过滤垃圾邮件

朴素贝叶斯的一个最著名应用：电子邮件垃圾过滤

1. 收集数据：提供文本文件  
2. 准备数据：将文本文件解析成词条向量  
3. 分析数据：检查词条确保解析的正确性  
4. 训练算法：使用之前建立的train_NB1函数  
5. 测试算法：使用classify_NB()函数，并且构建一个新的测试函数来计算文档集的错误率  
6. 使用算法：构建一个完整的程序对一组文档进行分类，将错分的文档输出到屏幕上

In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import re
import os

In [2]:
os.getcwd()
os.path.join(os.getcwd(),'probability_practice','email','spam')

'/Users/SeaMonster/PycharmProjects/MLIA-practice/probability_practice/email/spam'

之前建立的函数，搬过来

In [3]:
def train_NB1(train_matrix, train_category):
    """
    train_NB0函数的改进
    :param train_matrix:    训练文档矩阵（准确来说只是python原生二维数组）
    :param train_category:  训练文档矩阵对应的分类（一维向量）
    :return: (在非侮辱性文档类别下词汇表中单词的出现概率向量, 在侮辱性文档类别下词汇表中单词的出现概率向量, 任意文档属于侮辱性文档的概率)
    """
    # 训练文档的数目
    num_train_docs = len(train_matrix)
    num_words = len(train_matrix[0])

    # 分类为"侮辱性文档"的概率: 分类为"侮辱性文档"的文档数目，除以训练文档的总数目
    # train_category 为一维向量，只有0和1两种值，其中1代表侮辱性
    p_abusive = sum(train_category) / float(num_train_docs)

    # 初始化概率：初始化为1而不是0
    # p0_num: 对于分类0，词汇表中单词的出现次数
    # p1_num: 对于分类1，词汇表中单词的出现次数
    p0_num = np.ones(num_words)
    p1_num = np.ones(num_words)

    # 这两个是充当分母，
    # TODO 为什么是2？为什么不是1或者其他数？
    p0_denom = 2.0
    p1_denom = 2.0

    for i in range(num_train_docs):
        if train_category[i] == 1:  # 类别为"侮辱"
            p1_num += train_matrix[i]
            p1_denom += sum(train_matrix[i])  # 训练文档中的单词同时也在词汇表中存在的数目
        else:
            p0_num += train_matrix[i]
            p0_denom += sum(train_matrix[i])  # 训练文档中的单词同时也在词汇表中存在的数目

    # 概率向量(转为对数)
    # 分母是，对于分类1（侮辱性文档），一共出现过多少个单词（所有文档中）
    # 分子是，对于分类1（侮辱性文档），词汇表中每个单词各自出现的次数（所有文档中）
    # 相除后，得到一个对于分类1（侮辱性文档），词汇表中每个单词出现的概率
    p1_vector = np.log(p1_num / p1_denom)

    # 对每个元素做除法
    p0_vector = np.log(p0_num / p0_denom)

    return p0_vector, p1_vector, p_abusive

In [4]:
def classify_NB(vector_2_classify, p0_vector, p1_vector, p_class_1):
    """
    朴素贝叶斯分类函数

    这里边的公式都TMD怎么来的？！

    :param vector_2_classify:   要分类的向量
    :param p0_vector:           训练集中类别为"非侮辱性"的文档中，词汇表中每个单词的出现概率（组成一个一维向量）
    :param p1_vector:           训练集中类别为"侮辱性"的文档中，词汇表中每个单词的出现概率（组成一个一维向量）
    :param p_class_1:           训练集中类别为"侮辱性"文档的出现概率
    :return:
    """
    p1 = sum(vector_2_classify * p1_vector) + np.log(p_class_1)
    p0 = sum(vector_2_classify * p0_vector) + np.log(1.0 - p_class_1)
    print('p1:')
    print(p1)
    print('p0:')
    print(p0)
    if p1 > p0:
        return 1
    else:
        return 0

In [5]:
def create_vocabulary_list(data_set):
    """
    创建一个包含在所有文档中出现的不重复词的列表
    :param data_set:
    :return:
    """
    # 创建一个空集
    vocabulary_set = set([])
    for document in data_set:
        # 创建两个集合的并集
        vocabulary_set = vocabulary_set | set(document)
    return list(vocabulary_set)


def set_of_words_2_vector(vocabulary_list, input_set):
    """
    生成词集向量
    :param vocabulary_list:     词汇的列表
    :param input_set:           某个文档
    :return:                    文档向量，向量的每一个元素为1或0，分别表示词汇表中的单词在输入文档中是否出现
    """
    # 创建一个其中所含元素都为0的向量
    return_vector = [0]*len(vocabulary_list)
    for word in input_set:
        if word in vocabulary_list:
            return_vector[vocabulary_list.index(word)] = 1
        else:
            print('单词：%s 不在词汇表中！' %word)
    return return_vector

In [12]:
def text_parse(big_string):
    list_of_tokens = re.split(r'\w', big_string)
    return [tok.lower() for tok in list_of_tokens if len(tok)>2]


def spam_test():
    doc_list = []
    class_list = []
    full_text = []
    for i in range(1,26):
        # 导入并解析文本文件
        file_name = '%d.txt' %i
        file_full_path = \
            os.path.join(os.getcwd(),'probability_practice','email','spam',file_name)
        with open(file_full_path) as fr:
            word_list = text_parse(fr.read())
            doc_list.append(word_list)
            full_text.append(word_list)
            class_list.append(1)
        
        file_full_path = \
            os.path.join(os.getcwd(),'probability_practice','email','ham',file_name)
        with open(file_full_path) as fr:
            word_list = text_parse(fr.read())
            doc_list.append(word_list)
            full_text.append(word_list)
            class_list.append(0)
    vocabulary_list = create_vocabulary_list(doc_list)
    training_set = list(range(50))
    test_set= []
    
    # 随机构建训练集
    for i in range(10):
        random_index = int(np.random.uniform(0, len(training_set)))
        test_set.append(training_set[random_index])
        del(training_set[random_index])
        
    train_mat = []
    train_classes = []
    for doc_index in training_set:
        train_mat.append(set_of_words_2_vector(vocabulary_list, doc_list[doc_index]))
        train_classes.append(class_list[doc_index])
    
    prob_0_vector, prob_1_vector, prob_spam = train_NB1(
        np.array(train_mat),
        np.array(train_classes)
    )    
    
    error_count = 0
    
    # 对测试集分类
    for doc_index in test_set:
        word_vector = set_of_words_2_vector(vocabulary_list, doc_list[doc_index])
        if classify_NB(np.array(word_vector), prob_0_vector, prob_1_vector, prob_spam) \
            != class_list[doc_index]:
            error_count +=1
    print('the error rate is:', float(error_count)/len(test_set))
        

In [14]:
spam_test()

p1:
-3.14233574775
p0:
-4.02874727974
p1:
-16.7737732159
p0:
-14.0641350341
p1:
-12.0732928501
p0:
-9.29345040965
p1:
-3.14233574775
p0:
-4.02874727974
p1:
-8.75910684541
p0:
-6.85352175515
p1:
-4.75177366018
p0:
-7.70081961553
p1:
-4.75177366018
p0:
-2.77598431124
p1:
-6.45652175242
p0:
-8.7994319042
p1:
-11.3801456695
p0:
-12.1838221675
p1:
-3.14233574775
p0:
-4.02874727974
the error rate is: 0.0
